In [3]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import cv2
import os
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt


In [4]:
def load_data(dir_path, img_size=(512,512)):
    """
    Load resized images as np.arrays to workspace
    """
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels

TRAIN_DIR = 'H:/data/dataset_v1.500S/training/'
TEST_DIR = 'H:/data/dataset_v1.500S/test/'
VAL_DIR = 'H:/data/dataset_v1.500S/validation/'
IMG_SIZE = (512,512)

In [5]:
# use predefined function to load the image data into workspace
X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)
X_val, y_val, _ = load_data(VAL_DIR, IMG_SIZE)

100%|██████████| 4/4 [00:16<00:00,  4.11s/it]
ipykernel_launcher:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 4/4 [00:02<00:00,  1.71it/s]213 images loaded from H:/data/dataset_v1.500S/validation/ directory.



In [6]:
import imutils
def crop_imgs(set_name, add_pixels_value=0):
    """
    Finds the extreme points on the image and crops the rectangular out of them
    """
    set_new = []
    for img in set_name:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)

        # threshold the image, then perform a series of erosions +
        # dilations to remove any small regions of noise
        thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=2)

        # find contours in thresholded image, then grab the largest one
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)

        # find the extreme points
        extLeft = tuple(c[c[:, :, 0].argmin()][0])
        extRight = tuple(c[c[:, :, 0].argmax()][0])
        extTop = tuple(c[c[:, :, 1].argmin()][0])
        extBot = tuple(c[c[:, :, 1].argmax()][0])

        ADD_PIXELS = add_pixels_value
        new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
        k= 3
        e=0
        d=0
        g=16
        new_img1 = cv2.GaussianBlur(new_img, (k, k), g)
        new_img1 = cv2.erode(new_img1, None, iterations=e)
        new_img1 = cv2.dilate(new_img1, None, iterations=d)
        new_img1 = cv2.resize(
                    new_img1,
                    dsize=(512,512),
                    interpolation=cv2.INTER_CUBIC)
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        new_img1 = cv2.filter2D(new_img1, -1, kernel)
        set_new.append(new_img1)

    return np.array(set_new)
    

In [7]:
# apply this for each set
X_train_crop = crop_imgs(set_name=X_train)
X_val_crop = crop_imgs(set_name=X_val)
X_test_crop = crop_imgs(set_name=X_test)


In [8]:
# def preprocess_imgs(set_name, img_size):
    
#     set_new = []
#     for img in set_name:
#         img = cv2.resize(
#             img,
#             dsize=(512,512),
#             interpolation=cv2.INTER_CUBIC
#         )
#         set_new.append(img)
#     return np.array(set_new)
    


In [9]:
# X_train_prep = preprocess_imgs(set_name=X_train_crop, img_size=IMG_SIZE)
# X_test_prep = preprocess_imgs(set_name=X_test_crop, img_size=IMG_SIZE)
# X_val_prep = preprocess_imgs(set_name=X_val_crop, img_size=IMG_SIZE)
# print("Terkompile")

In [10]:
def save_new_images(x_set, y_set, folder_name):
    i = 0
    for (img, imclass) in zip(x_set, y_set):
        if imclass == 0:
            cv2.imwrite(folder_name+'glioma_tumor/glioma-'+str(i)+'.jpg', img)
        elif imclass == 1:
            cv2.imwrite(folder_name+'meningioma_tumor/meningioma-'+str(i)+'.jpg', img)
        elif imclass == 2:
            cv2.imwrite(folder_name+'no_tumor/normal-'+str(i)+'.jpg', img)
        elif imclass == 3:
            cv2.imwrite(folder_name+'pituitary_tumor/pituitary-'+str(i)+'.jpg', img)
        print("Gambar ke = ",i," tersimpan")
        i += 1


In [11]:
save_new_images(X_train_crop, y_train, folder_name='H:/data/dataset_v1.500P/training/')
save_new_images(X_test_crop, y_test, folder_name='H:/data/dataset_v1.500P/test/')
save_new_images(X_val_crop, y_val, folder_name='H:/data/dataset_v1.500P/validation/')

 tersimpan
Gambar ke =  1423  tersimpan
Gambar ke =  1424  tersimpan
Gambar ke =  1425  tersimpan
Gambar ke =  1426  tersimpan
Gambar ke =  1427  tersimpan
Gambar ke =  1428  tersimpan
Gambar ke =  1429  tersimpan
Gambar ke =  1430  tersimpan
Gambar ke =  1431  tersimpan
Gambar ke =  1432  tersimpan
Gambar ke =  1433  tersimpan
Gambar ke =  1434  tersimpan
Gambar ke =  1435  tersimpan
Gambar ke =  1436  tersimpan
Gambar ke =  1437  tersimpan
Gambar ke =  1438  tersimpan
Gambar ke =  1439  tersimpan
Gambar ke =  1440  tersimpan
Gambar ke =  1441  tersimpan
Gambar ke =  1442  tersimpan
Gambar ke =  1443  tersimpan
Gambar ke =  1444  tersimpan
Gambar ke =  1445  tersimpan
Gambar ke =  1446  tersimpan
Gambar ke =  1447  tersimpan
Gambar ke =  1448  tersimpan
Gambar ke =  1449  tersimpan
Gambar ke =  1450  tersimpan
Gambar ke =  1451  tersimpan
Gambar ke =  1452  tersimpan
Gambar ke =  1453  tersimpan
Gambar ke =  1454  tersimpan
Gambar ke =  1455  tersimpan
Gambar ke =  1456  tersimpan
Gam